# Juntando dados: concat e join {.unlisted .unnumbered}

Nas últimas aulas vimos as formas de processar e extrair informações de conjuntos de dados com DataFrame a partir do pandas. Isso inclui ferramentas poderosas que podem alterar a unidade de análise para gerar resultados mais complexos, como o `groupby` e o uso de regex. Em muitos casos precisamos ser capazes também de juntar conjuntos diferentes para um mesmo objeto, seja para adicionar informações que foram disponibilizadas separadamente, seja porque criamos informações adicionais que desejamos juntar ao conjunto original.

Nesses casos, precisaremos usar os métodos de `merge` e `concat` do pandas. Neste notebook, vamos explorar esses métodos e entender como eles podem ser usados para juntar conjuntos de dados diferentes.

## Material de apoio

1. [Postagem sobre como juntar dados no pandas usando merge e concat (em português).](https://medium.com/data-hackers/pandas-combinando-data-frames-com-merge-e-concat-10e7d07ca5ec)
2. Documentação do pandas para [merge](https://pandas.pydata.org/docs/reference/api/pandas.merge.html#pandas.merge) e [concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) são valiosas e incluem exemplos básicos de cada um.

# Caso da Aula

Na aula de hoje vamos assumir um projeto com informações de criação de novas leis pelo congresso nacional.


Sabendo que a Câmara dos Deputados [mantém um site de dados abertos](https://dadosabertos.camara.leg.br/), o que podemos fazer?

Navegando pelo site da Câmara, podemos testar e explorar várias coisas. Isso inclui várias formas de obter dados vias API, tópico de um dos arquivos de Apêndice de nossa apostila. 

Temos uma aba para arquivos, que fornce uma série de dados já compilados com diferentes períodos de atualização com informação sobre diversos aspectos do funcionamento do Congresso. Se olharmos com calma, conseguiremos baixar arquivos referente a anos inteiros.

Mas o que vamos fazer? Poderíamos explorar dados sobre a atuação de parlamentares, e seus gastos, ou acompanhar processos legislativos de alguma área do direito que podemos considerar relevante.

Vamos optar aqui pelo segundo caminho, e explorar projetos de lei. Imagine que fomos contratados por um escritório interessado em monitorar alterações legais sobre Direito do Consumidor.

Nosso trabalho será identificar os deputados responsáveis por essas propostas.

# Como proceder

[Nessa parte do site](https://dadosabertos.camara.leg.br/swagger/api.html#staticfile) já temos algumas instruções de uma forma fácil de conseguir diversos arquivos: podemos montar URLs com o ano do qual queremos obter as informações. Como o pandas sabe baixar arquivos de uma URL podemos passar diretamente o link pelo python para baixar os dados.

A informação da qual iremos partir está na parte "Classificação temática das proposições".

<blockquote>
Caminho para download: http://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/{formato}/proposicoesTemas-{ano}.{formato}, em que:

    {ano} é o ano de apresentação das proposições
    {formato} pode ser “csv”, “xlsx”, "ods", “json” ou “xml”.


</blockquote>

Como estamos trabalhando com o pandas, o formato mais amigável é o csv!

Para testar primeiro vamos carregar os dados de 2022:

Obs: O dicionário header é necessário pois, caso contrário, o servidor bloqueia nosso pedido de dados. Assim passamos esse dicionário para o argumento `storage_options` do método `read_csv`. Você não precisa se preocupar com isso, é um detalhe técnico que não é o foco da aula.


In [4]:
import time
import pandas as pd

formato = "csv"
lista_dados = list()

# esse código evita que o site bloqueio o acesso
header = {'User-Agent': 'Mozilla/5.0'}
url = f"http://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/{formato}/proposicoesTemas-2022.{formato}"

df_2022 = pd.read_csv(url, sep=";", storage_options=header)

df_2022.head(5)

,uriProposicao,siglaTipo,numero,ano,codTema,tema,relevancia
0,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,618,2022,58,Trabalho e Emprego,0
1,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,2253,2022,43,Direito Penal e Processual Penal,0
2,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,1367,2022,66,"Indústria, Comércio e Serviços",0
3,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,1367,2022,56,Saúde,0
4,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,3062,2022,48,Meio Ambiente e Desenvolvimento Sustentável,0


Outro detalhe importante que alteramos aqui foi o argumento `sep=";"`. Precisamos disso devido ao formato como o arquivo CSV está escrito. Na prática, isso significa que ele foi escrito com os campos separados por ponto e vírgula, e não por vírgula. Isso é muito comum para dados em português especialmente no Brasil.

Vamos trabalhar com os dados entre 2018 e 2023, para isso vamos usar um range para gerar a lista de números.

Convenientemente a url já usa o mesmo formato de format string do Python! Lembrando, format strings nos permitem inserir variáveis dentro de strings.

Para isso adicionamos o `f` antes da string, e dentro dela usamos chaves `{}` para indicar onde a variável será inserida.

Veja um exemplo mais simples:

```python
ano = 2022
print(f"O ano é {ano}")
```

Para coletar todos os anos podemos usar o código a seguir:

In [ ]:
import time
import pandas as pd

formato = "csv"
anos = range(2018,2024)
lista_dados = list()

# esse código evita que o site bloqueio o acesso
header = {'User-Agent': 'Mozilla/5.0'}

for ano in anos:
    url = f"http://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/{formato}/proposicoesTemas-{ano}.{formato}"
    print("Baixando", ano)
    _df_ano = pd.read_csv(url, sep=";", storage_options=header)
    lista_dados.append(_df_ano)
    # vamos esperar alguns segundos para evitar que o servidor bloqueie nosso acesso
    time.sleep(2.2)

Pronto conseguimos baixar os dados. Nós sabemos que eles estão corretos porque testamos enquanto baixávamos o material, você pode usar a célula abaixo para se certificar e ver como estão os dados.

Perceba que guardamos todos os dataframes em uma lista.

In [14]:
type(lista_dados)

list

Mas agora temos uma lista com vários dataframes que queremos juntar em apenas um. Como podemos fazer isso?

Para isso temos a função `.concat` mencionado no início da aula! Aqui **temos vários DataFrames dos quais queremos juntar as linhas em uma única estrutura**. Perceba que esperamos que eles tenham as mesmas colunas, afinal são recortes diferentes das mesmas observações.

Essa é a função principal do concat: consolidar as linhas de vários dataframes em uma única variável. Para isso podemos **passar um iterável (e.g. uma lista) com vários DataFrames para a função**. Ora, convenientemente já preparamos eles em uma lista.

In [15]:
df = pd.concat(lista_dados)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86249 entries, 0 to 21330
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uriProposicao  86249 non-null  object
 1   siglaTipo      86249 non-null  object
 2   numero         86249 non-null  int64 
 3   ano            86249 non-null  int64 
 4   codTema        86249 non-null  int64 
 5   tema           86249 non-null  object
 6   relevancia     86249 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 5.3+ MB


Conseguimos criar um único dataframe com 64 mil linhas!

Vamos descobrir os possíveis temas

In [16]:
df.tema.unique()

array(['Administração Pública', 'Finanças Públicas e Orçamento',
       'Cidades e Desenvolvimento Urbano', 'Economia', 'Saúde',
       'Energia, Recursos Hídricos e Minerais',
       'Arte, Cultura e Religião',
       'Relações Internacionais e Comércio Exterior',
       'Agricultura, Pecuária, Pesca e Extrativismo',
       'Meio Ambiente e Desenvolvimento Sustentável',
       'Homenagens e Datas Comemorativas', 'Trabalho e Emprego',
       'Comunicações', 'Direitos Humanos e Minorias', 'Educação',
       'Viação, Transporte e Mobilidade',
       'Previdência e Assistência Social',
       'Direito e Defesa do Consumidor', 'Esporte e Lazer',
       'Defesa e Segurança', 'Política, Partidos e Eleições',
       'Indústria, Comércio e Serviços',
       'Direito Penal e Processual Penal',
       'Direito Civil e Processual Civil', 'Estrutura Fundiária',
       'Ciência, Tecnologia e Inovação',
       'Processo Legislativo e Atuação Parlamentar', 'Direito e Justiça',
       'Turismo', 'Dire

Temos um bom número de temas... bom podemos filtrar diretamente pelo nome do tema ou usar o codTema referemte a Direito do Consumidor e filtrar por essa coluna.

In [17]:
df_consumidor = df.query("tema == 'Direito e Defesa do Consumidor'")

df_consumidor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1727 entries, 32 to 21212
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uriProposicao  1727 non-null   object
 1   siglaTipo      1727 non-null   object
 2   numero         1727 non-null   int64 
 3   ano            1727 non-null   int64 
 4   codTema        1727 non-null   int64 
 5   tema           1727 non-null   object
 6   relevancia     1727 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 107.9+ KB


Com isso diminuímos em muito nosso número de observações!


Agora vamos olhar um pouco para as informações que temos

In [18]:
df_consumidor.head()

,uriProposicao,siglaTipo,numero,ano,codTema,tema,relevancia
32,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,9471,2018,67,Direito e Defesa do Consumidor,0
54,https://dadosabertos.camara.leg.br/api/v2/prop...,PDC,884,2018,67,Direito e Defesa do Consumidor,0
97,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,9511,2018,67,Direito e Defesa do Consumidor,0
100,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,9512,2018,67,Direito e Defesa do Consumidor,0
110,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,9517,2018,67,Direito e Defesa do Consumidor,0


Agora precisamos capturar as informações sobre os autores da proposta... Poderíamos usar o link que conspicuamente aparece em cada uma das linhas. Mas como o link parece indicar, precisamos saber lidar com APIs para isso e fazer requisições individuais.

Temos outra opção... repetir o que fizemos com esses dados iniciais para a informação do autor de cada proposta na seção do site "Autores das Proposições por ano de apresentação". O que precisamos saber para baixar:

<blockquote>
Caminho para download: 

    http://dadosabertos.camara.leg.br/arquivos/proposicoesAutores/{formato}/proposicoesAutores-{ano}.{formato}, em que:

    {ano} é o ano de apresentação das proposições
    {formato} pode ser “csv”, “xlsx”, "ods", “json” ou “xml”.
</blockquote>


*Como já definimos variáveis com o formato e os anos acima podemos reutilizá-los, mas isso pressupõe que a célula tenha sido executada*

In [19]:
lista_anos = list()

# esse código evita que o site bloqueio o acesso
header = {'User-Agent': 'Mozilla/5.0'}

for ano in anos:
    url_autores = f"http://dadosabertos.camara.leg.br/arquivos/proposicoesAutores/{formato}/proposicoesAutores-{ano}.{formato}"
    _ano_autores = pd.read_csv(url_autores, sep=";", storage_options=header)
    lista_anos.append(_ano_autores)

prop_anos = pd.concat(lista_anos)

prop_anos.head()

,idProposicao,uriProposicao,idDeputadoAutor,uriAutor,codTipoAutor,tipoAutor,nomeAutor,siglaPartidoAutor,uriPartidoAutor,siglaUFAutor,ordemAssinatura,proponente
0,2338914,https://dadosabertos.camara.leg.br/api/v2/prop...,74057.0,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado(a),Alice Portugal,PCdoB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,1,1
1,2338666,https://dadosabertos.camara.leg.br/api/v2/prop...,133439.0,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado(a),André Figueiredo,PDT,https://dadosabertos.camara.leg.br/api/v2/part...,CE,1,1
2,2338661,https://dadosabertos.camara.leg.br/api/v2/prop...,73531.0,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado(a),Ivan Valente,PSOL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,1,1
3,2338660,https://dadosabertos.camara.leg.br/api/v2/prop...,178832.0,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado(a),Leandre,PSD,https://dadosabertos.camara.leg.br/api/v2/part...,PR,1,1
4,2338658,https://dadosabertos.camara.leg.br/api/v2/prop...,178987.0,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado(a),Orlando Silva,PCdoB,https://dadosabertos.camara.leg.br/api/v2/part...,SP,1,1


Agora precisamos juntar as duas informações, mas não queremos simplesmente "somar" as observações, queremos de fato cruzar informações, expandindo nosso tabela para o lado.

Para isso vamos usar a **função/o método merge que executa a operação de Joins no pandas**.

Para fazer um **join**, nós precisamos de **uma coluna com o mesmo significado em duas tabelas diferentes**. Ao informar o Pandas que aquela é a coluna pela qual iremos fazer o join, podemos fazer algumas previsões sobre as dimensões e as características da tabela resultante.

Essa coluna **pode ter o mesmo nome ou não**, ou poderia usar **múltiplas colunas**. No último caso, usaríamos a mesma lógica de selecionar colunas e do groupby: basta passar uma lista com o nome das colunas.

No nosso caso, estranhamento não tivemos o id das proposições incluídos na tabela com os temas, mas a coluna `uriProposicao` com os links, e ela contém link para as informações de cada prosição individualmente (inclusive o id). Dessa forma, podemos usá-la para o join.

In [ ]:
info_propostas = pd.merge(df_consumidor, prop_anos, left_on="uriProposicao", right_on="uriProposicao")

# em nosso caso poderíamos usar o fato de que o nome da coluna é igual
info_propostas = pd.merge(df_consumidor, prop_anos, on="uriProposicao")

info_propostas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2010 entries, 0 to 2009
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   uriProposicao      2010 non-null   object 
 1   siglaTipo          2010 non-null   object 
 2   numero             2010 non-null   int64  
 3   ano                2010 non-null   int64  
 4   codTema            2010 non-null   int64  
 5   tema               2010 non-null   object 
 6   relevancia         2010 non-null   int64  
 7   idProposicao       2010 non-null   int64  
 8   idDeputadoAutor    1947 non-null   float64
 9   uriAutor           2010 non-null   object 
 10  codTipoAutor       2010 non-null   int64  
 11  tipoAutor          2010 non-null   object 
 12  nomeAutor          2010 non-null   object 
 13  siglaPartidoAutor  1973 non-null   object 
 14  uriPartidoAutor    1820 non-null   object 
 15  siglaUFAutor       1973 non-null   object 
 16  ordemAssinatura    2010 

O que aconteceu que passamos a ter mais linhas agora do que tinhamos antes?

Provavelmente uma mesma proposta pode ter mais de um autor, assim como nossos DataFrames estavam com unidades diferentes passamos para a unidade mais geral.

Em termos técnicos nosso merge foi "one-to-many" porque adicionamos informações de várias linhas com uma mesma linha das propostas.

#### Diferentes tipos de join

Até agora, falamos em joins como se fossem uma única coisa, mas na verdade podemos unir tabelas de diferentes modos:

<br>
<br>
    <center> 
    <img src=https://miro.medium.com/v2/resize:fit:900/1*yb76Gk03pZsjVDp79n2yKA.jpeg style = "width:50%">
    </center>
    <!-- > fonte: https://medium.com/@rslavanyageetha/joins-in-pandas-6d95a2ba8a74 -->
<br>
<br>

A figura acima usa *diagramas de Venn* para representar os diferentes tipos de join que podem ser realizados.

É mais fácil compreender esses conceitos com exemplos.

Imagine que estamos trabalhando com 2 tabelas com dados do STF e que a tabela da direita está incompleta. Imagine, por exemplo, que o STF, em resposta a um pedido, nos fornece uma tabela contendo apenas aquelas ações de controle concentrado onde houve pelo menos uma ausência de ministro na sessão de julgamento.

A tabela da esquerda, nesse caso, poderia ter 10.000 linhas e 4 colunas, enquanto a da direita 5.000 linhas e 3 colunas ([como um grupo, os ministros faltam bastante](https://www.jota.info/opiniao-e-analise/colunas/coluna-do-jota-dados/ao-menos-dois-ministros-estao-ausentes-em-60-das-decisoes-em-plenario-do-stf-16022018 
)).

Um **inner join** retorna apenas a interseção entre as duas tabelas. Assim, as informações que estão em apenas uma das duas tabelas **são descartadas**. Por padrão, o Pandas faz inner joins. No exemplo do STF, o resultado teria 5.000 linhas e 6 colunas.

Por sua vez, a ideia de um **left join** é pegar **todas as colunas e linhas** da tabela da esquerda e juntar a ela apenas as **colunas e linhas** da tabela da direita que encontram alguma correspondência na tabela da esquerda.

No exemplo, se fizermos um **left join**, vamos terminar com uma tabela com 10.000 linhas e 6 colunas (afinal, uma das colunas é a mesma entre as duas tabelas). Para 5.000 das 10.000 linhas, porém, as informações a respeito das faltas não estarão preenchidas. Ou seja, elas estarão cheias de informações que são `Null` (SQL) ou `NaN` (Python/Pandas).

A função `pd.merge()` aceita um argumento opcional chamado `how`, que aceita como opções, dentre outras, o padrão "inner" ou "left".


Vamos alterar esse argumento para "left" e ver o que acontece com as dimensões do nosso dataframe. Em nosso caso específico esperamos que nada ocorra.

Além disso, vamos simplificar o merge já que ambas as colunas tem o mesmo nome, podemos usar o argumento `on=`

In [ ]:
info_propostas = pd.merge(df_consumidor, prop_anos, on="uriProposicao", how='left')

info_propostas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2846 entries, 0 to 2845
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   uriProposicao      2846 non-null   object 
 1   siglaTipo          2846 non-null   object 
 2   numero             2846 non-null   int64  
 3   ano                2846 non-null   int64  
 4   codTema            2846 non-null   int64  
 5   tema               2846 non-null   object 
 6   relevancia         2846 non-null   int64  
 7   idProposicao       2846 non-null   int64  
 8   idDeputadoAutor    2774 non-null   float64
 9   uriAutor           2846 non-null   object 
 10  codTipoAutor       2846 non-null   int64  
 11  tipoAutor          2846 non-null   object 
 12  nomeAutor          2846 non-null   object 
 13  siglaPartidoAutor  2813 non-null   object 
 14  uriPartidoAutor    2553 non-null   object 
 15  siglaUFAutor       2813 non-null   object 
 16  ordemAssinatura    2846 

Agora podemos seguir com nossa análise: quais os deputados com maior número de propostas nesse tema?

Para isso basta agruparmos por deputado e contarmos quantas propostas únicas cada um deles fez.

In [21]:
info_propostas.groupby("nomeAutor")['idProposicao'].nunique().sort_values(ascending=False).head(10)

nomeAutor
Capitão Alberto Neto    44
Alexandre Frota         39
Edna Henrique           33
Léo Moraes              33
Eduardo da Fonte        31
Weliton Prado           29
Felipe Carreras         24
José Nelto              24
Juninho do Pneu         24
Célio Studart           23
Name: idProposicao, dtype: int64

Temos os 10 deputados com maior número de autoria nesse período.

Poderíamos desenvolver essa análise em diversos aspectos, como: expandir temporalmente, análisar os status de andamento de cada proposição. Por exemplo, ver qual o deputado com maior número de propostas aprovadas.

## Outra forma de usar o Join

Em aulas anteriores, falamos sobre o groupby.

Uma outra forma de usar o merge seria juntar informações que nós mesmos calculamos a partir de multiplos groupbys.

Fatalmente seria mais simples usarmos o agg, mas como tudo com programação podemos tomar múltiplos caminhos para chegar ao mesmo resultado.

Vamos fazer uma análise por UF assim.

In [ ]:
info_propostas.columns

Index(['uriProposicao', 'siglaTipo', 'numero', 'ano', 'codTema', 'tema',
       'relevancia', 'idProposicao', 'idDeputadoAutor', 'uriAutor',
       'codTipoAutor', 'tipoAutor', 'nomeAutor', 'siglaPartidoAutor',
       'uriPartidoAutor', 'siglaUFAutor', 'ordemAssinatura', 'proponente'],
      dtype='object')

In [23]:
# numero de deputados por estado
n_deputados = info_propostas.groupby("siglaUFAutor", as_index=False)["idDeputadoAutor"].nunique()

# numero propostas
# outra forma de fazer esse groupby
n_props = info_propostas.groupby("siglaUFAutor")["idProposicao"].nunique().reset_index()

df_estados = pd.merge(n_deputados, n_props, on="siglaUFAutor")

df_estados


,siglaUFAutor,idDeputadoAutor,idProposicao
0,AC,14,58
1,AL,14,33
2,AM,15,96
3,AP,10,30
4,BA,41,111
5,CE,33,135
6,DF,14,47
7,ES,16,52
8,GO,21,75
9,MA,28,82


A forma mais "pythonica" de chegar no mesmo resultado seria:

In [24]:
df_estados = info_propostas.groupby("siglaUFAutor").agg(
    {
        "idDeputadoAutor": "nunique",
        "idProposicao": "nunique",
    }
)


## Resumo

Nesta aula, aprendemos sobre como juntar dados usando o concat e o merge usando um exemplo de dados da câmara dos deputados.

- A função `pd.concat` nos permite juntar DataFrames somando suas linhas

- A função/método `pd.merge` nos permite cruzar dados de múltiplas fontes, ou apenas armazenados em DataFrames diferentes se tivermos uma informação que identifica o mesmo caso nos dois conjuntos.

# Caso da aula de Regex

Terminamos a aula sobre expressões regulares também com um problema de juntar dados. Havíamos coletado os processos citados nas ementas de decisões relacionadas a liberdade de expressão.

Você pode agora voltar ao exemplo e entender melhor o que foi feito para juntar os dados. Naquele exemplo, precisamos usar os índices do pandas para juntar os dados, em vez de uma ou mais colunas.

# Exercício

Para praticar o que aprendemos, você deve coletar parte dos dados que acabmos de apresentar novamente. Então, realizar algumas análises que dependem também das informações do conjunto de proposições.

Colete as informações para o ano de 2018 apenas. **Para cada tema**, calcule:

1. Quantos deputados diferentes apresentaram propostas
2. Quantos partidos diferentes apresentaram propostas; e
3. Quantas propostas foram apresentadas nesse ano.